In [8]:
from datetime import datetime
from django.db.models import Sum
from decimal import Decimal

fa = FinancialAccount.objects.first()
date_start = datetime.fromisoformat('2022-01-04T16:41:24+02:00')
date_end = datetime.fromisoformat('2024-01-04T16:41:24+02:00')

date_filters = {}
if date_start:
    date_filters["transaction_time__gte"] = date_start
if date_end:
    date_filters["transaction_time__lte"] = date_end
            
txs = fa.transactions.filter(**date_filters)


txs.annotate(
    calculated_balance= \
        Sum('amount', filter=Q(transaction_type='CREDIT')) -
        Sum('amount', filter=Q(transaction_type='DEBIT'))
)

total = Decimal(0)
for a in txs.values("currency_code", "transaction_type").annotate(total=Sum("amount")):
    total += (-1 if a["transaction_type"] == "DEBIT" else 1) * \
        Transaction.amount_to_USD(a["total"], a["currency_code"])
    
    
print(total)
    
txs.values("currency_code", "transaction_type").annotate(total=Sum("amount"))

# fa.recalculate_balance()
fa.recalculate_balance()
fa.save()

6592.0000
